<a href="https://colab.research.google.com/github/azzahyumnafaiza/Market-Basket-Analysis/blob/main/Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Impor Packages

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

# Import Data

In [ ]:
from google.colab import drive
drive.mount ("/content/MyDrive")

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/MyDrive/MyDrive/Online Retail Data (1).csv")
df.head()

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


# Data Cleaning

In [ ]:
#Mengidentifikasi isi kolom tipe data object (apakah order date bisa dijadikan date_time)

for dtype, col in (list(zip(df.dtypes, df.columns))):
  if dtype == "object":
    print(col, dtype)
    print(df[col].unique())

  else :
    print(col, dtype)
    print(df[col].min(), df[col].max())

  print()



order_id object
['493410' 'C493411' '493412' ... '539990' '539991' '539992']

product_code object
['TEST001' '21539' '21724' ... '22721' '22970' '22971']

product_name object
['This is a test product.' 'RETRO SPOTS BUTTER DISH'
 'PANDA AND BUNNIES STICKER SHEET' ... '*USB Office Mirror Ball'
 'MINI LADLE LOVE HEART PINK' 'MINI LADLE LOVE HEART RED']

quantity int64
-9600 10200

order_date object
['2010-01-04 09:24:00' '2010-01-04 09:43:00' '2010-01-04 09:53:00' ...
 '2010-12-23 16:31:00' '2010-12-23 16:49:00' '2010-12-23 17:41:00']

price float64
-53594.36 25111.09

customer_id float64
12346.0 18287.0



In [ ]:
#Mengubah order date menjadi date_time
df_dc = df.copy()
#df_dc["order_date"] = df_dc["order_date"].astype("datetime64[ns]")
df_dc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


In [ ]:
#cleaning
df_dc = df.copy()
#membuat kolom date
df_dc["date"] = pd.to_datetime(df_dc["order_date"]).dt.date.astype("datetime64[ns]")
#menghapus baaris yang tidak memiliki customer_id
df_dc = df_dc[~df_dc["customer_id"].isna()]
#mengahpus baris yg tidak ada product name
df_dc = df_dc[~df_dc["product_name"].isna()]
#menyamakan nama produk menjadi huruf kecil
df_dc["product_name"] = df_dc["product_name"].str.lower()
#menghapus semua baris dengan product_code atau product_name dengan nama "test"
df_dc = df_dc[(~df_dc["product_code"].str.lower().str.contains("test")) |
              (~df_dc["product_name"].str.contains("test")) ]
#membuat kolom order_status dengan nilai "cancelled" jika order id diawali "c"
#membuat kolom order_status dengan nilai "delivered" jika order_id selain hruuf "c"
df_dc["order_status"] = np.where(df_dc["order_id"].str.startswith("C"), "cancelled", "delivered")
#menghapus data dengan order status cancled
df_dc = df_dc[df_dc["order_status"] == "delivered"]
#mengubah nilai qty menjadi positif semua,
df_dc["quantity"] = df_dc["quantity"].abs()
#menghapus baris yang memiliki nilai harga negatif, memasukan nilai yang positif saja
df_dc = df_dc[df_dc["price"] > 0]
#membuat nilai amount
df_dc["amount"] = df_dc["quantity"] * df_dc["price"]
# mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu product_name-nya yang paling sering muncul
most_freq_product_name = df_dc.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_dc = df_dc.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_dc['product_name'] = df_dc['most_freq_product_name']
df_dc = df_dc.drop(columns='most_freq_product_name')
#mengubah cust_id menjadi str
df_dc["customer_id"] = df_dc["customer_id"].astype("str")


<ipython-input-7-9bd6c629a8a8>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dc["quantity"] = df_dc["quantity"].abs()
<ipython-input-7-9bd6c629a8a8>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dc["amount"] = df_dc["quantity"] * df_dc["price"]


In [ ]:
#menghapus outlier
from scipy import stats
df_dc = df_dc[(np.abs(stats.zscore(df_dc[["quantity", "amount"]])) < 3).all(axis=1)]
df_dc = df_dc.reset_index(drop=True)
df_dc

,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,order_status,amount
0,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,91.80
1,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,51.00
2,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,5.10
3,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,8.50
4,493414,21527,red retrospot traditional teapot,12,2010-01-04 10:28:00,6.95,14590.0,2010-01-04,delivered,83.40
...,...,...,...,...,...,...,...,...,...,...
350087,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,2010-12-23,delivered,1.25
350088,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,delivered,1.69
350089,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,delivered,1.69
350090,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,2010-12-23,delivered,9.90


In [ ]:
df_dc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350092 entries, 0 to 350091
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      350092 non-null  object        
 1   product_code  350092 non-null  object        
 2   product_name  350092 non-null  object        
 3   quantity      350092 non-null  int64         
 4   order_date    350092 non-null  object        
 5   price         350092 non-null  float64       
 6   customer_id   350092 non-null  object        
 7   date          350092 non-null  datetime64[ns]
 8   order_status  350092 non-null  object        
 9   amount        350092 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 26.7+ MB


# Membuat DF Basket


In [ ]:
df_basket = pd.pivot_table(df_dc, index="order_id", columns="product_name", values="product_code", aggfunc="nunique", fill_value =0)
df_basket

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539982,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Membuat Encode Df basket

In [ ]:
def encode (x) :
  if x == 0 :
    return False
  if x > 0 :
    return True

In [ ]:
df_basket_encode = df_basket.applymap(encode)
df_basket_encode

<ipython-input-12-fe6f76d658e7>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_basket_encode = df_basket.applymap(encode)


product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539981,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539985,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
df_basket_filter = df_basket_encode[(df_basket_encode>0).sum(axis=1)>1]
df_basket_filter

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539978,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539981,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Algoritma Apriori

In [ ]:
# Mencari item set dan support
!pip install mlxtend==0.21.0

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

freq_item = apriori(df_basket_filter, min_support=0.01, use_colnames=True).sort_values("support", ascending=False).reset_index(drop=True)
freq_item["product_cnt"] = freq_item["itemsets"].apply(lambda x: len(x))
freq_item

,support,itemsets,product_cnt
0,0.177953,(white hanging heart t-light holder),1
1,0.099440,(regency cakestand 3 tier),1
2,0.096841,(jumbo bag red retrospot),1
3,0.079912,(pack of 72 retro spot cake cases),1
4,0.078512,(assorted colour bird ornament),1
...,...,...,...
1189,0.010064,"(key fob , back door , key fob , front door ,...",3
1190,0.010064,"(wrap,suki and friends)",1
1191,0.010064,(retrospot padded seat cushion),1
1192,0.010064,"(pack of 6 birdy gift tags, pack of 6 panneton...",2


In [ ]:
#hitung nilai support, confidince, dan lift
product_association = association_rules(freq_item, metric="confidence", min_threshold=0.7).sort_values(["support", "confidence"], ascending=[False, False]).reset_index(drop=True)
product_association

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(red hanging heart t-light holder),(white hanging heart t-light holder),0.058784,0.177953,0.042455,0.722222,4.058510,0.031995,2.959371
1,(sweetheart ceramic trinket box),(strawberry ceramic trinket box),0.049254,0.074980,0.037457,0.760487,10.142533,0.033764,3.862089
2,(toilet metal sign),(bathroom metal sign),0.026993,0.040589,0.021728,0.804938,19.831353,0.020632,4.918499
3,(red retrospot sugar jam bowl),(red retrospot small milk jug),0.023660,0.037123,0.016796,0.709859,19.121592,0.015917,3.318652
4,(painted metal pears assorted),(assorted colour bird ornament),0.021927,0.078512,0.016596,0.756839,9.639738,0.014874,3.789618
...,...,...,...,...,...,...,...,...,...
81,"(green 3 piece mini dots cutlery set, blue 3 p...",(pink 3 piece mini dots cutlery set),0.011464,0.029992,0.010064,0.877907,29.271370,0.009720,7.944827
82,"(green 3 piece mini dots cutlery set, pink 3 p...",(blue 3 piece mini dots cutlery set),0.011997,0.030525,0.010064,0.838889,27.481853,0.009698,6.017430
83,"(key fob , back door , key fob , front door )","(key fob , shed)",0.012463,0.025060,0.010064,0.807487,32.222153,0.009752,5.064272
84,"(poppy's playhouse kitchen, poppy's playhouse ...",(poppy's playhouse bathroom),0.013730,0.010997,0.010064,0.733010,66.655016,0.009913,3.704266


# Analisis

1. Produk Red Hanging Heart T-light Holder dibeli 5.9% dari total transaksi
2. Saat pembeli sudah membeli Red Hanging Heart T-light Holder sebesar 72% pembeli akan membeli White Hanging Heart T-light Holder juga.
3. Terdapat asosiasi yang sangat kuat antara R dan W dengan peluang kedua produk dibeli bersama dalam 1x transaksi adalah 4x lebih besar dari peluang kedua produk dibeli secara masing masing
4. Berdasarkan insight tsb, produk W dapat ditempatkan di dekat atau sebelah R atau kedua produk dapa
